<a href="https://colab.research.google.com/github/rexad/NLP_notebooks/blob/master/Kaggle/jigsaw/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#logistic regression with Scikit-learn

##Importing our libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import gensim.downloader as api
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import os.path
from google.colab import drive

##Load data for jigsaw 

In [4]:
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/"
train = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Data/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


TIMEOUT: ignored

##Feature engineering

In [ ]:
train